### Student Information
Name: 蕭靖澂 Ching-Cheng Hsiao

Student ID: 110033632

GitHub ID: hsiaooo

Kaggle name: chingchenghsiao

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

In [1]:
### Begin Assignment Here

### 1. Load kaggle dataset

In [2]:
# load the libraries
import os
import json
import pandas as pd
import numpy as np

# load the json
path       = os.path.dirname(os.path.abspath("tweets_DM.json"))
raw_data = pd.read_json(path+"/dataset/tweets_DM.json",lines=True)
tweets   = pd.json_normalize(data=raw_data['_source'])
identify = pd.read_csv(path+"/dataset/data_identification.csv")
emotion  = pd.read_csv(path+"/dataset/emotion.csv")

In [3]:
tweets.head()

,tweet.hashtags,tweet.tweet_id,tweet.text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [4]:
emotion_list = []
id_list = []
for i, j in zip(emotion['emotion'], emotion['tweet_id']):
    if ',' in i:
        i = i.split(',')[1]
    emotion_list.append(i)
    id_list.append(j)
print(emotion_list[:50])
print(id_list[:50])
emotion_dict = {'tweet_id':id_list, 'emotion':emotion_list}
emotion_2 = pd.DataFrame(emotion_dict)

['sadness', 'disgust', 'anticipation', 'joy', 'anticipation', 'joy', 'sadness', 'sadness', 'trust', 'sadness', 'anticipation', 'joy', 'sadness', 'joy', 'joy', 'joy', 'joy', 'joy', 'sadness', 'disgust', 'trust', 'anticipation', 'sadness', 'anger', 'joy', 'trust', 'joy', 'fear', 'joy', 'sadness', 'joy', 'anticipation', 'trust', 'joy', 'joy', 'joy', 'disgust', 'joy', 'joy', 'joy', 'anticipation', 'sadness', 'disgust', 'joy', 'joy', 'trust', 'sadness', 'joy', 'anger', 'fear']
['0x3140b1', '0x368b73', '0x296183', '0x2bd6e1', '0x2ee1dd', '0x34cd80', '0x33f099', '0x2ae7b7', '0x2408d4', '0x2b193b', '0x215b2e', '0x349b83', '0x270297', '0x213b5e', '0x31a0e7', '0x28a0d1', '0x2c8770', '0x317e52', '0x30188c', '0x361411', '0x1e16a0', '0x255135', '0x203527', '0x27b6b2', '0x213263', '0x255bbd', '0x33a6b7', '0x30e5c4', '0x2bffa3', '0x1dfa03', '0x369a1e', '0x37db4e', '0x1c9da9', '0x1c91b2', '0x1d5ec5', '0x38b81e', '0x25e867', '0x383ed2', '0x1e539d', '0x346f76', '0x1d7368', '0x2b732b', '0x31bb2a', '0x1f8

In [5]:
emotion_2.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [6]:
# rename column names
tweets = tweets.rename(index=str, columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags"})

# add identify tags to dataframe
tweets = pd.merge(tweets, identify, on="tweet_id")

print(tweets.shape)
tweets.head()

(1867535, 4)


,hashtags,tweet_id,text,identification
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test


In [7]:
# remove emoji and punctuation
from string import punctuation
import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def remove_punctuation_emoji(value):
    value = remove_emojis(value)
    return ''.join([c for c in value if (c not in punctuation)])

In [ ]:
# remove the emoji, punctuation and LH, which in a lot of text, and make all word be lower
from tqdm import tqdm

for i, text in tqdm(enumerate(tweets['text'])):
    tweets['text'][i] = remove_punctuation_emoji(text).replace("LH", "").replace("  ", " ").lower()    

216502it [1:28:38, 37.84it/s]

In [ ]:
print(tweets['text'][3])

In [ ]:
# get training set and test set
train_df = tweets[tweets["identification"] == "train"]
test_df  = tweets[tweets["identification"] == "test"]

# drop identification tags
train_df.drop(columns=["identification"], inplace=True)
test_df.drop(columns=["identification"], inplace=True)

# drop hashtags
train_df.drop(columns=["hashtags"], inplace=True)
test_df.drop(columns=["hashtags"], inplace=True)

print(f'Training set\n{train_df.head()}\n')
print(f'Testing set\n{test_df.head()}')

In [ ]:
# add emotion column
train_df = pd.merge(train_df, emotion_2, on="tweet_id")
test_df = test_df.assign(emotion="")

In [ ]:
# use tweet_id as index
train_df.set_index("tweet_id",inplace=True)
test_df.set_index("tweet_id",inplace=True)

train_df.head()

In [ ]:
test_df.head()

In [ ]:
# save to pickle file
train_df.to_pickle("/pkl/train_df_clean.pkl")
test_df.to_pickle("/pkl/test_df_clean.pkl")

In [ ]:
# load a pickle file
train_df = pd.read_pickle(path+"/pkl/train_df_clean.pkl")
test_df  = pd.read_pickle(path+"/pkl/test_df_clean.pkl")

In [ ]:
print("Train_df\n", train_df)
print("\nTest_df\n", test_df)

### 2. Pre-processing

#### 2.1 Tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
# from nltk.tokenize import word_tokenize
# from nltk.tokenize.stanford import StanfordTokenizer

tknzr = TweetTokenizer(preserve_case=False)
# tknzr = word_tokenize()
# tknzr = StanfordTokenizer()
tfidf = TfidfVectorizer(max_features=50000, stop_words='english', tokenizer=tknzr.tokenize)

# fitting
tfidf.fit(train_df['text'])

*In Tf-idf, I used max_features=20000, 30000, 50000.*

In [ ]:
# transforming training sets
X_train = tfidf.transform(train_df['text'])
X_train.shape

In [ ]:
# transforming testing sets
X_test = tfidf.transform(test_df['text'])
X_test.shape

In [ ]:
# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [ ]:
print(y_test[0:4])

#### 2.2 Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# build analyzers (bag-of-words)
BOW_vectorizer = CountVectorizer() 

In [ ]:
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
BOW_vectorizer.fit(train_df['text'])

# 2. Transform documents to document-term matrix.
train_data_BOW_features = BOW_vectorizer.transform(train_df['text'])
test_data_BOW_features = BOW_vectorizer.transform(test_df['text'])

In [ ]:
# check the result
train_data_BOW_features

In [ ]:
type(train_data_BOW_features)

In [ ]:
# check the dimension
train_data_BOW_features.shape

In [ ]:
# observe some feature names
feature_names = BOW_vectorizer.get_feature_names_out()
feature_names[:10]

In [ ]:
import nltk

# build analyzers (bag-of-words)
BOW_20000 = CountVectorizer(max_features=20000, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_20000.fit(train_df['text'])

train_data_BOW_features_20000 = BOW_20000.transform(train_df['text'])

## check dimension
train_data_BOW_features_20000.shape

In [ ]:
# observe some feature names
feature_names_20000 = BOW_20000.get_feature_names()
feature_names_20000[100:110]

### 3. Train model

#### 3.1 Logistic regression

In [ ]:
# copy the X_train, X_test, y_train, y_test for Logistic regression
lr_X_train = X_train
lr_X_test  = X_test
lr_y_train = y_train
lr_y_test  = y_test

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=6, n_jobs=-1, max_iter=1000)
lr.fit(lr_X_train,lr_y_train)

In [ ]:
pred_result_lr = lr.predict(lr_X_test)
pred_result_lr.shape

In [ ]:
import pickle
f = open('model/lr_tfidf50000_clean.pickle', 'wb')
pickle.dump(lr, f)
f.close()

In [ ]:
# save the result
test_df['emotion']=pred_result_lr
test_df.drop(columns=['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/lr_tfidf50000_clean.csv')
print('Saved file to ', path, '/submission/lr_tfidf50000_clean.csv')

I used the GridSearchCV to find the best parameters of Logistic Regression, and the result is C = 10000, penalty = "l2", but if I use C=1000, penalty="l2", submit to kaggle, the score only 0.45819, less than use C=6, score is 0.45963.

#### 3.2 XGBoost Classifier

In [ ]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df = pd.read_pickle(path+"/test_df.pkl")

# copy the X_train, X_test, y_train, y_test for XGBoost
xgb_X_train = X_train
xgb_X_test  = X_test
xgb_y_train = y_train
xgb_y_test  = y_test

In [ ]:
emotion = pd.Categorical(xgb_y_train.iloc[:], categories=xgb_y_train.iloc[:].unique(), ordered=True)

xgb_y_train.iloc[:] = emotion.codes
print(xgb_y_train)
print('anger \t\t= ', xgb_y_train[9])
print('anticipation \t= ', xgb_y_train[0])
print('disgust \t= ', xgb_y_train[18])
print('fear \t\t= ', xgb_y_train[2])
print('joy \t\t= ', xgb_y_train[3])
print('sadness \t= ', xgb_y_train[1])
print('surprise \t= ', xgb_y_train[19])
print('trust \t\t= ', xgb_y_train[15])

In [ ]:
import xgboost as xgb

xgbc = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgbc.fit(xgb_X_train, xgb_y_train)

In [ ]:
pred_result_xgbc = xgbc.predict(xgb_X_test)

pred_result_xgbc.shape
print(pred_result_xgbc)

In [ ]:
print(pred_result_xgbc[-10:])

In [ ]:
pred_result = []
for i, emo in enumerate(pred_result_xgbc):
    if emo == 0:
        pred_result.append('anticipation')
    elif emo == 1:
        pred_result.append('sadness')
    elif emo == 2:
        pred_result.append('fear')
    elif emo == 3:
        pred_result.append('joy')
    elif emo == 4:
        pred_result.append('anger')
    elif emo == 5:
        pred_result.append('trust')
    elif emo == 6:
        pred_result.append('disgust')
    elif emo == 7:
        pred_result.append('surprise')

In [ ]:
pred = np.array(pred_result)
print(pred[-10:])

In [ ]:
f = open('model/xgbc_tfid_clean.pickle', 'wb')
pickle.dump(xgbc, f)
f.close()

In [ ]:
# save the result
test_df['emotion'] = pred
test_df.drop(columns = ['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns = ['emotion']
test_df.to_csv(path+'/submission/xgbc_clean.csv')
print('Saved file to ', path, '/submission/xgbc_clean.csv')

#### 3.3 Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
DT_X_train = X_train
DT_y_train = y_train
DT_X_test = X_test
DT_y_test = y_test

# take a look at data dimension 
print('X_train.shape: ', DT_X_train.shape)
print('y_train.shape: ', DT_y_train.shape)
print('X_test.shape: ', DT_X_test.shape)
print('y_test.shape: ', DT_y_test.shape)

In [ ]:
emotion = pd.Categorical(DT_y_train.iloc[:], categories=DT_y_train.iloc[:].unique(), ordered=True)

DT_y_train.iloc[:] = emotion.codes
DT_y_train = DT_y_train.astype('int')
print(DT_y_train)

In [ ]:
# build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

# training!
DT_model = DT_model.fit(DT_X_train, DT_y_train)

# predict!
y_test_pred = DT_model.predict(DT_X_test)

# so we get the pred result
y_test_pred[:10]

In [ ]:
f = open('model/DT_tfidf_clean.pickle', 'wb')
pickle.dump(DT_model, f)
f.close()

In [ ]:
# save the result
test_df['emotion']= y_test_pred
test_df.drop(columns=['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/DT_tfidf_cleancsv')
print('Saved file to ', path, '/submission/DT_tfidf_clean.csv')

#### 3.4 LSTM

In [ ]:
from keras.models import Model
from keras.utils import np_utils
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import rmsprop_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [ ]:
# load a pickle file
train_df = pd.read_pickle(path+"/pkl/train_df.pkl")
test_df = pd.read_pickle(path+"/pkl/test_df.pkl")

# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [ ]:
max_words = 20000
max_len = 300
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train_df['text'])

In [ ]:
train_seq = tok.texts_to_sequences(train_df['text'])
test_seq = tok.texts_to_sequences(test_df['text'])

train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(test_seq_mat.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
# y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

In [ ]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

In [ ]:
inputs = Input(name='inputs',shape=[max_len])
## Embedding
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dropout(0.5)(layer)
layer = Dense(output_shape,activation="softmax",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=rmsprop_v2.RMSprop(),metrics=["accuracy"])

In [ ]:
model_fit = model.fit(train_seq_mat, y_train,batch_size=128, epochs=3, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] )

#### *Train epoch 5 times:*
Epoch 1/5</br>
11372/11372 [==============================] - 3087s 271ms/step - loss: 1.2539 - accuracy: 0.5520</br>
Epoch 2/5</br>
11372/11372 [==============================] - 3091s 272ms/step - loss: 1.1417 - accuracy: 0.5952</br>
Epoch 3/5</br>
11372/11372 [==============================] - 3088s 272ms/step - loss: 1.1145 - accuracy: 0.6066</br>
Epoch 4/5</br>
11372/11372 [==============================] - 3088s 272ms/step - loss: 1.1034 - accuracy: 0.6127</br>
Epoch 5/5</br>
11372/11372 [==============================] - 3090s 272ms/step - loss: 1.0969 - accuracy: 0.6166</br>

*Acutally in epoch 5, the accuracy is decreasing gradually, it's mean that this result is overfitting, so I finally choose epoch 3 to training.*

In [ ]:
# predict the result using our model
pred_result_lstm = label_decode(label_encoder, model.predict(test_seq_mat, batch_size=128))
pred_result_lstm[:5]

In [ ]:
from keras.models import load_model
model.save('model_clean.h5')  # creates a HDF5 file 'model.h5'

In [ ]:
# save the result
test_df['emotion']=pred_result_lstm
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/keras_tfidf_clean.csv')
print('Saved file to ', path, '/submission/keras_tfidf_clean.csv')

#### 3.6 BERT

#### Play with BERTTokenizer

<small><i>*You can safely skip this section if you're already familar with BERTTokenizer.</i></small>

Let's play with this tokenizer a little bit before we go on.

Using this tokenizer is pretty easy: just call this object, and it processes the sentences for you.  

In [ ]:
import os

# select your GPU. Note that this should be set before you load tensorflow or pytorch.
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# To use multiple GPUs, combine all GPU ID with commas
# e.g. >>> os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,3'

In [ ]:
import torch
# Check if any GPU is used
torch.cuda.is_available()

In [ ]:
# the model you want to use. Available models can be found here: https://huggingface.co/models
#MODEL_NAME = 'distilbert-base-uncased'
MODEL_NAME = 'bert-base-uncased'

Load data

In [ ]:
# for a classificaiton problem, you need to provide both training & testing data
BERT_X_train = X_train
BERT_y_train = y_train
BERT_X_test  = X_test
BERT_y_test  = y_test

# take a look at data dimension 
print('X_train.shape: ', BERT_X_train.shape)
print('y_train.shape: ', BERT_y_train.shape)
print('X_test.shape: ', BERT_X_test.shape)
print('y_test.shape: ', BERT_y_test.shape)

Preprocessing

In [ ]:
from transformers import AutoTokenizer # For tokenization

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# First, declare a new encoder
encoder = OneHotEncoder(sparse = False)
# Then, let the encoder learns all features in the given dataset
# Keep in mind that all `fit` functions in sklearn only make the encoder learn from the data, not transforming the data yet.
encoder = encoder.fit(np.reshape(dataset['train']['emotion'], (-1, 1)))